#Relational Data Persistence and Schema Definition
The primary objective for this module is to take the curated dataset generated in the previous step and persist it into a relational SQLite database.

##Loading Cleaned Dataset
We import the clean version of the soybean production dataset (soja_produccion_clean.csv), which contains 12,470 validated records. This ensures that only data that has passed our agronomic and statistical quality checks is persisted into the database.

In [1]:
import pandas as pd
import sqlite3

df_clean = pd.read_csv("soja_produccion_clean.csv")


## Database Connection

In this section, we establish a connection to soja_produccion.db. To optimize the data structure, we normalize the flat file into three distinct relational tables:
1. production: The main fact table containing yearly yields and surfaces.
2. provinces: A dimension table for unique province identification.
3. departments: A dimension table for granular administrative units, linked via provincia_id

To avoid redundancy, we extract unique identifiers and names for provinces and departments.

In [2]:
conn = sqlite3.connect("soja_produccion.db")

df_clean.to_sql(
    "production",
    conn,
    if_exists="replace",
    index=False
)

provinces = (
    df_clean[['provincia_id', 'provincia_nombre']]
    .drop_duplicates()
)

provinces.to_sql(
    "provinces",
    conn,
    if_exists="replace",
    index=False
)

departments = (
    df_clean[['departamento_id', 'departamento_nombre', 'provincia_id']]
    .drop_duplicates()
)

departments.to_sql(
    "departments",
    conn,
    if_exists="replace",
    index=False
)

355

## Data Integrity Verification
As a final quality gate, we perform a SQL query to verify that the total row count in the production table matches our source file (12,470 rows). Once the integrity is confirmed, the connection is safely closed to prevent leaks

In [3]:
pd.read_sql("SELECT COUNT(*) FROM production", conn)


,COUNT(*)
0,12470


In [4]:
conn.close()